# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [91]:
import numpy as np
from sklearn import datasets, tree, metrics, ensemble
from sklearn.cross_validation import train_test_split

boston = datasets.load_boston()

X = boston.data
y = boston.target

#(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = 0.25)
X_train = X[:X.shape[0]*0.75,:]
X_test = X[X.shape[0]*0.75:,:]
y_train = y[:X.shape[0]*0.75]
y_test = y[X.shape[0]*0.75:]

C:\Users\pavel.dudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\pavel.dudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\pavel.dudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\pavel.dudukin\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [92]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [93]:
def write_answer(ans,i):
    with open("GB_answer"+str(i)+".txt", "w") as fout:
        fout.write(str(ans))

In [94]:
base_algorithms_list = []
coefficients_list = []

y_current = np.array(y_train)
num_trees = 50

for i in range(num_trees):
    DTR_2 = tree.DecisionTreeRegressor(max_depth = 5, random_state = 42)
    DTR_2_res = DTR_2.fit(X_train, y_current)
    base_algorithms_list.append(DTR_2_res)
    coefficients_list.append(0.9)
    y_current = y_train - gbm_predict(X_train)

In [97]:
RMSE_2 = np.sqrt(metrics.mean_squared_error(y_test, gbm_predict(X_test)))
print(RMSE_2)
write_answer(RMSE_2,2)

5.47665097417


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 9). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [98]:
base_algorithms_list = []
coefficients_list = []

y_current = np.array(y_train)
num_trees = 10

for i in range(num_trees):
    DTR_3 = tree.DecisionTreeRegressor(max_depth = 5, random_state = 42)
    DTR_3_res = DTR_3.fit(X_train, y_current)
    base_algorithms_list.append(DTR_3_res)
    coefficients_list.append(0.9/(1 + i))
    y_current = y_train - gbm_predict(X_train)

In [99]:
RMSE_3 = np.sqrt(metrics.mean_squared_error(y_test, gbm_predict(X_test)))
print(RMSE_3)
write_answer(RMSE_3,3)

4.65090865256


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [100]:
depth = [1, 3, 10, 50, 75, 100, 200, 300, 500]
trees = [1, 3, 10, 50, 75, 100, 200, 300, 500]

for d in depth:
    for t in trees:
        GBR = ensemble.GradientBoostingRegressor(n_estimators = t, max_depth = d, random_state = 42)
        GBR_res = GBR.fit(X_train,y_train)
        GBR_pred = GBR.predict(X_test)
        print("Depth = "+str(d)+"; Trees = "+str(t)+"; RMSE = "+
              str(np.sqrt(metrics.mean_squared_error(y_test, GBR_pred))))

Depth = 1; Trees = 1; RMSE = 11.2217086156
Depth = 1; Trees = 3; RMSE = 10.8233790868
Depth = 1; Trees = 10; RMSE = 9.48835769365
Depth = 1; Trees = 50; RMSE = 6.48599849325
Depth = 1; Trees = 75; RMSE = 5.82028927943
Depth = 1; Trees = 100; RMSE = 5.67029454046
Depth = 1; Trees = 200; RMSE = 6.31992210672
Depth = 1; Trees = 300; RMSE = 6.56497970819
Depth = 1; Trees = 500; RMSE = 6.73999404151
Depth = 3; Trees = 1; RMSE = 10.7951646845
Depth = 3; Trees = 3; RMSE = 9.68445208172
Depth = 3; Trees = 10; RMSE = 7.45190494934
Depth = 3; Trees = 50; RMSE = 4.9208321567
Depth = 3; Trees = 75; RMSE = 4.69845796809
Depth = 3; Trees = 100; RMSE = 4.60625535388
Depth = 3; Trees = 200; RMSE = 4.41887696256
Depth = 3; Trees = 300; RMSE = 4.3868432527
Depth = 3; Trees = 500; RMSE = 4.36575484593
Depth = 10; Trees = 1; RMSE = 10.8173895971
Depth = 10; Trees = 3; RMSE = 9.75701217282
Depth = 10; Trees = 10; RMSE = 7.38002536765
Depth = 10; Trees = 50; RMSE = 5.99258304904
Depth = 10; Trees = 75; RMSE

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [101]:
from sklearn import linear_model

estimator = linear_model.LinearRegression()
LReg = estimator.fit(X_train, y_train)
LReg_predictions = estimator.predict(X_test)

RMSE5 = np.sqrt(metrics.mean_squared_error(y_test, LReg_predictions))
print(RMSE5)

write_answer(RMSE5,5)


8.27046803494
